In [16]:
import xml.etree.ElementTree as ET
import json

def parse_complexType(complexType_element):
    json_schema = {
        "type": "object",
        "properties": {}
    }
    
    sequence = complexType_element.find('{http://www.w3.org/2001/XMLSchema}sequence')
    if sequence is not None:
        for element in sequence.findall('{http://www.w3.org/2001/XMLSchema}element'):
            json_schema["properties"][element.attrib['name']] = handle_element(element)

    return json_schema

def handle_element(element):
    if 'type' in element.attrib:
        element_type = element.attrib['type']
        json_schema = {}
        if element_type.startswith('xsd:'):
            if element_type == "xsd:string":
                json_schema["type"] = "string"
            elif element_type == "xsd:int":
                json_schema["type"] = "int"
            elif element_type in ["xsd:long", "xsd:decimal"]:
                json_schema["type"] = "number"
            elif element_type == "xsd:base64Binary":
                json_schema = {
                    "type": "string",
                    "binaryEncoding ": "base64"
                }
            elif element_type == "xsd:boolean":
                   json_schema["type"] = "booelan"
            elif element_type == "xsd:dateTime":
                json_schema = {
                    "type": "string",
                    "format": "date-time"
                }
            elif element_type == "xsd:date":
                json_schema = {
                    "type": "string",
                    "format": "date"
                }
        else:
            json_schema["type"] = "object"
        
        return json_schema
    else:
        complexType = element.find('{http://www.w3.org/2001/XMLSchema}complexType')
        if complexType is not None:
            return parse_complexType(complexType)
        else:
            return {
                "type": "object",
                "properties": {}
            }

def xsd_to_json_schema(xsd,json_file):
    tree = ET.parse(xsd)
    root = tree.getroot()

    json_schema = {
        "$schema": "http://json-schema.org/draft-07/schema#",
        "type": "object",
        "properties": {}
    }

    for element in root.findall('{http://www.w3.org/2001/XMLSchema}element'):
        json_schema["properties"][element.attrib['name']] = handle_element(element)

    with open(json_file, 'w') as f:
        json.dump(json_schema, f, indent=2)




In [17]:
# usage
xsd_file='c:\\users\\inbra\\workspaceUtil\\PersonProfileSchemas_Custom.xsd'
output_json='c:\\users\\inbra\\workspaceUtil\\PersonProfileSchemas_Custom.json'
xsd_to_json_schema(xsd_file, output_json)